# Morningstar Pro - Entraînement avancé sur Colab

## Système complet de trading algorithmique avec données sociales

Ce notebook permet :
- De télécharger les données de marché (OHLCV) depuis un exchange crypto
- D'ajouter des indicateurs techniques avancés
- D'intégrer des données sociales (GitHub et Reddit) si les APIs sont configurées
- D'entraîner un modèle de deep learning pour le trading

## 1. Installation des dépendances

In [ ]:
# Installation des dépendances système et Python
# Utilisation de versions spécifiques pour TF, NumPy, Pandas
!pip install -q tensorflow==2.12.0 pandas==1.5.3 numpy==1.23.5
!pip install -q ccxt==4.1.91 ta pyarrow scikit-learn asyncpraw tweepy aiohttp PyGithub praw nest_asyncio jedi

# --- IMPORTANT : Redémarrez MANUELLEMENT l'environnement d'exécution après cette cellule --- #
# (Menu Exécution > Redémarrer l'environnement d'exécution)
print("Dépendances installées. Veuillez redémarrer l'environnement d'exécution.")

## 2. Configuration

In [ ]:
# Clonage du dépôt et ajout du chemin Morningstar
# Assurez-vous que le kernel a été redémarré avant cette cellule
import os
if not os.path.exists('/content/eva001'):
    !git clone https://github.com/Cabrel10/eva001.git
else:
    print("Dépôt déjà cloné.")
import sys
if '/content/eva001' not in sys.path:
    sys.path.insert(0, '/content/eva001')
print("Chemin ajouté.")

In [ ]:
# Sélection interactive des paires et des dates
import datetime
default_pairs = 'BTC/USDT,ETH/USDT,BNB/USDT,SOL/USDT'
pairs_input = input(f"Entrez les paires séparées par une virgule (exemple: {default_pairs}): ") or default_pairs
pairs = [p.strip().upper() for p in pairs_input.split(',')]
start_date = input("Date de début (YYYY-MM-DD, défaut 2023-01-01): ") or '2023-01-01'
end_date = input("Date de fin (YYYY-MM-DD, défaut aujourd'hui): ") or str(datetime.date.today())
print(f"Paires sélectionnées: {pairs}")
print(f"Période: {start_date} à {end_date}")

In [ ]:
# Configuration des APIs sociales (Optionnel)
try:
    from github import Github
    import praw
except ImportError:
    print("Bibliothèques GitHub/Reddit non trouvées. Installation...")
    !pip install -q PyGithub praw
    from github import Github
    import praw

# Config GitHub (optionnel)
github_token = input("Entrez votre token GitHub (laissez vide pour ignorer): ") or None
gh = None
if github_token:
    try:
        gh = Github(github_token)
        gh.get_user().login # Teste la connexion
        print("GitHub API configurée.")
    except Exception as e:
        print(f"Erreur configuration GitHub: {e}. Les données GitHub ne seront pas collectées.")
        gh = None
else:
    print("Token GitHub non fourni. Les données GitHub ne seront pas collectées.")

# Config Reddit (optionnel)
reddit_client_id = input("Reddit client_id (laissez vide pour ignorer): ") or None
reddit_client_secret = input("Reddit client_secret (laissez vide pour ignorer): ") or None
reddit = None
if reddit_client_id and reddit_client_secret:
    try:
        reddit = praw.Reddit(
            client_id=reddit_client_id,
            client_secret=reddit_client_secret,
            user_agent="Morningstar Data Collector by Cline",
            read_only=True # Mode lecture seule important
        )
        # Test simple pour vérifier la connexion
        print(f"Test de connexion Reddit pour l'utilisateur: {reddit.user.me()}")
        print("Reddit API configurée (mode lecture seule).")
    except Exception as e:
        print(f"Erreur configuration ou test Reddit: {e}. Les données Reddit ne seront pas collectées.")
        reddit = None
else:
    print("Credentials Reddit non fournis. Les données Reddit ne seront pas collectées.")

## 3. Pipeline de données

In [ ]:
# Fonctions pour données sociales
def get_github_stats(repo_name):
    if not gh:
        return 0, 0, 0, 0, 0 # Retourner 0 si non configuré
    try:
        repo = gh.get_repo(repo_name)
        # Utiliser des méthodes qui ne requièrent pas d'itération complète si possible
        commits_count = repo.get_commits().totalCount # Peut être lourd
        stars_count = repo.stargazers_count
        forks_count = repo.forks_count
        open_issues_count = repo.get_issues(state='open').totalCount
        closed_issues_count = repo.get_issues(state='closed').totalCount
        return commits_count, stars_count, forks_count, open_issues_count, closed_issues_count
    except Exception as e:
        print(f"Erreur GitHub pour {repo_name}: {e}")
        return 0, 0, 0, 0, 0 # Retourner 0 en cas d'erreur

def get_reddit_sentiment(subreddit, pair):
    if not reddit:
        return 0.5 # Retourner neutre si non configuré
    try:
        # Recherche plus ciblée
        query = f'({pair.split("/")[0]} OR {pair.replace("/","")}) flair:Discussion'
        submissions = reddit.subreddit(subreddit).search(query, limit=20, sort='relevance', time_filter='month')
        scores = [s.score for s in submissions]
        if not scores:
             return 0.5 # Retourner une valeur neutre si pas de posts
        positive_ratio = sum(1 for s in scores if s > 0) / len(scores)
        return positive_ratio
    except Exception as e:
        print(f"Erreur Reddit pour {subreddit} / {pair}: {e}")
        return 0.5 # Retourner neutre en cas d'erreur API

In [ ]:
# Téléchargement des données de marché
from Morningstar.utils.data_manager import ExchangeDataManager
import pandas as pd
import asyncio
import nest_asyncio
import numpy as np # Importer numpy ici
from datetime import datetime, timedelta # Importer datetime ici

nest_asyncio.apply()

async def fetch_data(pairs, timeframe='1h', start_date=None, end_date=None):
    exchange = ExchangeDataManager(exchange_name="kucoin") # Utiliser Kucoin
    await exchange.load_markets_async()
    all_data = []
    tasks = []
    for pair in pairs:
        print(f"Préparation du téléchargement {pair}...")
        tasks.append(exchange.load_data(pair, timeframe, start_date=start_date, end_date=end_date))
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    await exchange.close()

    successful_data = []
    for i, result in enumerate(results):
        pair = pairs[i]
        if isinstance(result, pd.DataFrame) and not result.empty:
            print(f"Données {pair} téléchargées.")
            result['pair'] = pair
            successful_data.append(result)
        else:
            print(f"Échec ou données vides pour {pair}: {result}")
            
    if successful_data:
        return pd.concat(successful_data)
    else:
        # Ne pas lever d'erreur ici, retourner un DataFrame vide
        print("AVERTISSEMENT: Aucune donnée téléchargée pour aucune paire.")
        return pd.DataFrame() 

# Utiliser les variables globales pairs, start_date, end_date
raw_data = pd.DataFrame() # Initialiser à vide
try:
    raw_data = asyncio.get_event_loop().run_until_complete(fetch_data(pairs, '1h', start_date, end_date))
    if raw_data.empty:
        print("Aucune donnée téléchargée - vérifiez les paires et les dates. Création de données synthétiques.")
    else:
        print(f"Données brutes téléchargées : {raw_data.shape} lignes")
        print(raw_data.head())
except Exception as e:
    print(f"Erreur lors du téléchargement : {e}")

# Créer des données synthétiques si raw_data est vide
if raw_data.empty:
    print("Création d'un petit dataset synthétique pour continuer...")
    dates = pd.to_datetime([datetime.now() - timedelta(hours=i) for i in range(100)])
    synth_data = {
        'open': np.random.uniform(100, 200, 100),
        'high': np.random.uniform(100, 210, 100),
        'low': np.random.uniform(90, 200, 100),
        'close': np.random.uniform(100, 200, 100),
        'volume': np.random.uniform(1000, 5000, 100),
        'pair': 'SYNTH/USDT'
    }
    raw_data = pd.DataFrame(synth_data)
    raw_data['datetime'] = dates # Ajouter la colonne datetime
    raw_data = raw_data.set_index('datetime') # Définir l'index si nécessaire
    print("Dataset synthétique créé :", raw_data.shape)

In [ ]:
# Prétraitement et sauvegarde
from Morningstar.utils.custom_indicators import add_technical_indicators

def prepare_dataset(df):
    if df.empty:
        print("DataFrame d'entrée vide, impossible de préparer le dataset.")
        return df
        
    # S'assurer que l'index est DatetimeIndex si ce n'est pas déjà le cas
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'datetime' in df.columns:
             df['datetime'] = pd.to_datetime(df['datetime'])
             df = df.set_index('datetime')
        else:
             print("AVERTISSEMENT: Colonne 'datetime' manquante et index non DatetimeIndex.")
             # Tenter de réinitialiser si l'index n'est pas unique
             if not df.index.is_unique:
                  df = df.reset_index(drop=True)
    elif not df.index.is_unique:
         print("AVERTISSEMENT: L'index Datetime n'est pas unique, réinitialisation.")
         df = df.reset_index(drop=True) # Réinitialiser si DatetimeIndex mais non unique
         
    # Ajouter les indicateurs techniques
    try:
        df = add_technical_indicators(df.copy()) # Utiliser une copie
    except Exception as e:
        print(f"Erreur lors de l'ajout des indicateurs: {e}. Indicateurs ignorés.")
    
    # Initialiser les colonnes sociales
    social_cols = ['commits', 'stars', 'forks', 'issues_opened', 'issues_closed', 'reddit_sentiment']
    for col in social_cols:
        df[col] = 0.0 # Initialiser à 0.0 pour type numérique
        
    # Récupérer les données sociales pour chaque paire si API configurée
    repo_map = {
        'BTC/USDT': 'bitcoin/bitcoin',
        'ETH/USDT': 'ethereum/go-ethereum',
        'BNB/USDT': 'binance-chain/docs',
        'SOL/USDT': 'solana-labs/solana'
    }
    subreddit_map = {
        'BTC/USDT': 'Bitcoin',
        'ETH/USDT': 'ethereum',
        'BNB/USDT': 'binance',
        'SOL/USDT': 'solana'
    }
        
    if 'pair' in df.columns:
        for pair in df['pair'].unique():
            mask = df['pair'] == pair
            
            if gh and pair in repo_map:
                print(f"Récupération GitHub pour {pair}...")
                commits, stars, forks, issues_opened, issues_closed = get_github_stats(repo_map[pair])
                df.loc[mask, 'commits'] = commits
                df.loc[mask, 'stars'] = stars
                df.loc[mask, 'forks'] = forks
                df.loc[mask, 'issues_opened'] = issues_opened
                df.loc[mask, 'issues_closed'] = issues_closed
                
            if reddit and pair in subreddit_map:
                print(f"Récupération Reddit pour {pair}...")
                sentiment = get_reddit_sentiment(subreddit_map[pair], pair.split('/')[0])
                df.loc[mask, 'reddit_sentiment'] = sentiment
    else:
        print("Colonne 'pair' manquante, impossible de récupérer les données sociales.")
    
    # S'assurer que 'datetime' est une colonne pour la sélection finale
    if isinstance(df.index, pd.DatetimeIndex):
         df = df.reset_index() # Convertir l'index datetime en colonne
         
    final_columns = [
        'open', 'high', 'low', 'close', 'volume', 'rsi', 'macd', 'macd_signal', 'macd_hist',
        'bb_upper', 'bb_middle', 'bb_lower', 'volume_ma', 'volume_anomaly', 'pair',
        'commits', 'stars', 'forks', 'issues_opened', 'issues_closed', 'reddit_sentiment', 'datetime'
    ]
    
    # S'assurer que toutes les colonnes finales existent et remplir les NaN
    for col in final_columns:
        if col not in df.columns:
            df[col] = 0.0 # Initialiser à 0.0
            
    # Convertir les colonnes sociales en numérique et remplir NaN
    for col in social_cols:
         df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0)
         
    # Remplir les NaN restants (indicateurs techniques au début)
    df = df.fillna(method='ffill').fillna(0.0)
            
    return df[final_columns] # Retourner avec l'ordre final

data = prepare_dataset(raw_data.copy()) # Utiliser une copie

if not data.empty:
    data.to_parquet('full_dataset.parquet')
    print(f"Dataset final préparé: {data.shape}")
    print(data.info())
    print(data.head())
else:
    print("Le dataset est vide après préparation.")

## 4. Entraînement du modèle

In [ ]:
# Entraînement du modèle Morningstar
import tensorflow as tf
from Morningstar.workflows.training_workflow import TrainingWorkflow
import numpy as np

if data.empty:
    print("Dataset vide, impossible d'entraîner le modèle.")
else:
    print(f"\n=== Préparation des données pour TF ===")
    print(f"Taille du dataset Pandas: {len(data)} échantillons")

    # Configuration adaptative
    class ColabConfig:
        def __init__(self):
            self.time_window = min(50, max(10, len(data)//10)) 
            # Exclure les colonnes non numériques ou non pertinentes
            self.features = [col for col in data.columns if data[col].dtype in [np.float64, np.int64] and col not in ['datetime', 'pair', 'close']] 
            # S'assurer que 'close' n'est pas dans les features si c'est la cible
            # self.features = [col for col in data.columns if col not in ['datetime', 'pair', 'close']] 
            self.target_col = 'close' # Définir explicitement la cible
            self.epochs = 200
            self.batch_size = min(1024, max(32, len(data)//5))
            self.dataset_path = 'full_dataset.parquet'

    colab_config = ColabConfig()
    print(f"Configuration utilisée: {vars(colab_config)}")

    # Conversion en dataset TensorFlow avec vérification
    print("\n=== Conversion en dataset TensorFlow ===")
    workflow = TrainingWorkflow(colab_config)
    try:
        # Assurer que _prepare_dataset utilise self.features et self.target_col
        tf_dataset = workflow._prepare_dataset(data) 
        
        dataset_size = tf.data.experimental.cardinality(tf_dataset).numpy()
        print(f"Taille du dataset TensorFlow: {dataset_size} éléments (avant batch)")
        
        if dataset_size < colab_config.time_window + 1:
             raise ValueError(f"Dataset trop petit ({dataset_size}) pour la fenêtre temporelle ({colab_config.time_window})")
             
        # Batching après vérification de taille
        tf_dataset = tf_dataset.batch(colab_config.batch_size).prefetch(tf.data.AUTOTUNE)
        batched_size = tf.data.experimental.cardinality(tf_dataset).numpy()
        print(f"Taille du dataset TensorFlow après batching: {batched_size} batches")
        if batched_size < 1:
             raise ValueError("Le dataset TensorFlow est vide après batching")
             
    except Exception as e:
        print(f"\nERREUR lors de la conversion TF: {str(e)}")
        # Optionnel: arrêter ou continuer avec un modèle non entraîné
        raise e # Arrêter l'exécution

    # Division train/val
    print("\n=== Division train/validation ===")
    # Diviser AVANT le batching est plus simple si _prepare_dataset retourne un dataset non batché
    # Si _prepare_dataset retourne déjà batché, il faut unbatch/rebatch ou skip/take sur les batches
    val_batches = max(1, int(batched_size * 0.2))
    train_dataset = tf_dataset.skip(val_batches)
    val_dataset = tf_dataset.take(val_batches)

    print(f"Train batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
    print(f"Val batches: {tf.data.experimental.cardinality(val_dataset).numpy()}")
    
    # Vérifier que les datasets ne sont pas vides après division
    if tf.data.experimental.cardinality(train_dataset).numpy() == 0:
        raise ValueError("Le dataset d'entraînement est vide après division.")
    if tf.data.experimental.cardinality(val_dataset).numpy() == 0:
        print("Attention: Le dataset de validation est vide après division. Entraînement sans validation.")
        val_dataset = None

    # Construction du modèle
    print("\n=== Construction du modèle ===")
    with tf.distribute.MirroredStrategy().scope():
        input_shape = (colab_config.time_window, len(colab_config.features))
        print(f"Input shape du modèle: {input_shape}")
        inputs = tf.keras.Input(shape=input_shape)
        x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
        x = tf.keras.layers.LSTM(128)(x)
        x = tf.keras.layers.Dense(64, activation='swish')(x)
        outputs = tf.keras.layers.Dense(1)(x)
        model = tf.keras.Model(inputs, outputs)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='huber',
            metrics=['mae']
        )
    model.summary()

    # Callbacks
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss' if val_dataset else 'loss'),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, monitor='val_loss' if val_dataset else 'loss'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss' if val_dataset else 'loss', restore_best_weights=True)
    ]

    # Entraînement
    print("\n=== Début de l'entraînement ===")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=colab_config.epochs,
        callbacks=callbacks,
        verbose=1 # Afficher la progression
    )

## 5. Sauvegarde finale

In [ ]:
# Sauvegarde finale et export sur Google Drive
if 'model' in locals(): # Vérifier si le modèle a été entraîné
    model.save('morningstar_pro.h5')
    print("Modèle Keras sauvegardé localement.")
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        !mkdir -p '/content/drive/MyDrive/Colab Data/' # Créer le dossier si besoin
        !cp morningstar_pro.h5 '/content/drive/MyDrive/Colab Data/'
        print("Modèle copié sur Google Drive.")
    except Exception as e:
        print(f"Erreur lors de la copie sur Google Drive: {e}")
else:
    print("Aucun modèle n'a été entraîné (dataset vide ou erreur précédente).")